In [34]:
import os
os.chdir("C:\\Users\\19789\\Desktop\\SNHU\\data science\\Chicken Disease project (end to end)\\Chicken-Disease-Classification")
%pwd

'C:\\Users\\19789\\Desktop\\SNHU\\data science\\Chicken Disease project (end to end)\\Chicken-Disease-Classification'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path 

In [36]:
from src.ChickenDiseaseClassification.constants import *
from src.ChickenDiseaseClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [37]:
class ConfigurationManager:
  def __init__(
    self,
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])


  def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
      config = self.config.prepare_callbacks
      model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
      create_directories([
        Path(model_ckpt_dir),
        Path(config.tensorboard_root_log_dir)
      ])

      prepare_callback_config = PrepareCallbacksConfig(
        root_dir = Path(config.root_dir),
        tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
        checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
      ) 
      return prepare_callback_config


In [38]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [39]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only = True
        )
    
    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [40]:
try:
  config = ConfigurationManager()
  prepare_callbacks_config = config.get_prepare_callback_config()
  prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
  callback_list = prepare_callbacks.get_tb_ckpt_callback()

except Exception as e:
  raise e

[2024-11-08 10:43:28,688: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-08 10:43:28,690: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-08 10:43:28,690: INFO: common: created directory at: artifacts]
[2024-11-08 10:43:28,691: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-11-08 10:43:28,691: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [41]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-11-08-10-45-26'